In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

## creating function 'extract' which extracts features

In [22]:
def extract(soundfiles_dataset, mfcc, chroma, mel):
    with soundfile.SoundFile(soundfiles_dataset) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [23]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

## Function to Load data

In [24]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("soundfiles_dataset\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

## loading the features and splitting them into train and test data

In [25]:
x_train, x_test, y_train, y_test = load_data(test_size = 0.25)

In [29]:
print(x_train.shape[0], x_test.shape[0])

576 192


In [30]:
print('features extracted :', x_train.shape[1])

features extracted : 180


## Training the Model

In [79]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(500,), learning_rate='adaptive', max_iter=190)

In [80]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(500,),
              learning_rate='adaptive', max_iter=190)

In [81]:
y_pred=model.predict(x_test)

In [82]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#printing accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 70.83%
